# Having Fun with Recurrent Neural Networks (RNN)

**~$ whoami**

Sergio G. Burdisso (sergio.burdisso@gmail.com)

**~$ cat recommended_reading.txt**

[The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

## 1. Imports

In [1]:
import tensorflow as tf
import numpy as np

from tensorflow.contrib import layers
from tensorflow.contrib import rnn

from idataset import Dataset

## 2. Hyper-parameters

In [2]:
hStateSize = 512  # Number of Hidden Units (NHU)
maxSeqLength = 128  # MSL
nLayers = 3  # NL

learningRate = 1e-3  # 0.001
dropoutProb = 0.3

batchSize = 200  # BS
alphaSize = Dataset.get_alphabet_size()  # AS

## 3. Our model
### 1. Inputs

In [3]:
def tf_create_inputs():
    global X_train, y_train, h_state_input, dropout, batch_size

    X_train = tf.placeholder(tf.uint8, [None, None], name="X_train")  # (BS, MSL)
    # expected outputs = same sequence shifted by 1 since we are trying to predict the next character
    y_train = tf.placeholder(tf.uint8, [None, None], name="y_train")  # (BS, MSL)
    h_state_input = tf.placeholder(tf.float32, [None, hStateSize * nLayers], name="h_state_input")  # (BS, NHU*NL)


    dropout = tf.placeholder(tf.float32, name="dropout")
    batch_size = tf.placeholder(tf.int32, name="batch_size")

## 3. Our model
### 2. Architecture

In [4]:
def tf_create_architecture():
    global rnn_cells_stack_outdropout

    rnn_cells = [rnn.GRUCell(hStateSize) for _ in range(nLayers)]

    rnn_cells_indropout = [
        rnn.DropoutWrapper(cell, input_keep_prob=(1 - dropout))
        for cell in rnn_cells
    ]

    rnn_cells_stack = rnn.MultiRNNCell(rnn_cells_indropout, state_is_tuple=False)
    # dropout for the softmax layer

    rnn_cells_stack_outdropout = rnn.DropoutWrapper(
        rnn_cells_stack, output_keep_prob=(1 - dropout)
    )

## 3. Our model
### 3. TF Graph  (1/3) - Forward Propagation

In [5]:
def tf_graph_forward_propagation():
    global h_state_outputs, next_h_state_input, X_train_oh, y_train_oh

    X_train_oh = tf.one_hot(X_train, alphaSize, 1.0, 0.0)  # (BS, MSL, AS)
    y_train_oh = tf.one_hot(y_train, alphaSize, 1.0, 0.0)  # (BS, MSL, AS)

    h_state_outputs, next_h_state_input = tf.nn.dynamic_rnn(
        rnn_cells_stack_outdropout,
        X_train_oh,
        dtype=tf.float32,
        initial_state=h_state_input
    )
    # h_state_outputs.shape = (BS, MSL, NHU)
    # next_h_state_input.shape = (BS, NHU*NL), this is the last state in the sequence


    next_h_state_input = tf.identity(next_h_state_input, name='next_h_state_input')  # just to give it a name

## 3. Our model
### 3. TF Graph  (2/3) - Fully Connected Net + Softmax

In [6]:
def tf_graph_fullyconnected_softmax():
    global h_state_outputs_flat, y_preds, y_preds_prob
    # flatting
    h_state_outputs_flat = tf.reshape(h_state_outputs, [-1, hStateSize]) # (BS*MSL, NHU)

    y_preds = layers.fully_connected(  # (BS*MSL, AS)
        h_state_outputs_flat,
        alphaSize,
        activation_fn=tf.nn.relu # the default
        # activation_fn=tf.nn.softmax => WARNING: This op expects unscaled logits, since it performs a softmax on logits internally for efficiency. Do not call this op with the output of softmax, as it will produce incorrect results.
    )

    y_preds_prob = tf.nn.softmax(y_preds, name="y_preds_prob")  # (BS*MSL, AS)

## 3. Our model
### 3. TF Graph  (3/3) -  Training

In [7]:
def tf_graph_training():
    global y_train_oh_flat, loss, train_step
    # flatting
    y_train_oh_flat = tf.reshape(y_train_oh, [-1, alphaSize])  # (BS*MSL, AS)

    loss = tf.nn.softmax_cross_entropy_with_logits(logits=y_preds, labels=y_train_oh_flat)  # (BS*MSL)
    loss = tf.reshape(loss, [batch_size, -1])  # (BS, MSL)

    train_step = tf.train.AdamOptimizer(learningRate).minimize(loss)

## 4. Training (1/2)

In [8]:
def tf_init_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True # only needed when using GPU
    sess = tf.Session(config=config)
    sess.run(tf.global_variables_initializer())
    return sess
    

## 4. Training (2/2)

In [9]:
def train(pattern_path, trained_model_name, n_epochs=20):
    from time import time

    dataset = Dataset.load_from_files(pattern_path)
    sess = tf_init_session()
    n_chars_processed = 0
    oepoch = -1
    t_start = time()

    # initial input hidden state (zero)
    in_h_state = np.zeros([batchSize, hStateSize * nLayers])
    try:        
        print("[training] starting to learn :) ...")
        for epoch, X_train_batch, y_train_batch in Dataset.get_training_batches(
            n_epochs, batchSize, maxSeqLength, dataset
        ):

            _, next_in_h_state = sess.run(
                [train_step, next_h_state_input],
                feed_dict={
                    X_train: X_train_batch, y_train: y_train_batch, h_state_input: in_h_state,
                    dropout: dropoutProb, batch_size: batchSize
                }
            )
            in_h_state = next_in_h_state

            n_chars_processed += batchSize * maxSeqLength
            if oepoch != epoch:
                oepoch = epoch
                print(
                    "[training][epoch %d/%d] characters processed: %d" %
                    (epoch + 1, n_epochs, n_chars_processed)
                )
    except KeyboardInterrupt:
        pass

    print("[training] training finished (%.1f mins)" % ((time() - t_start) / 60))
    saved_file = tf.train.Saver().save(sess, 'trained_models/test/%s' % trained_model_name)
    print("[training] model saved: " + saved_file)

## 4. Sequence Generator

In [15]:
def generate_seq(model_path, seq_start="a", length=1000):
    tf.reset_default_graph()

    output_seq = seq_start    
    sess = tf_init_session()
    with sess:
        try:
            saver = tf.train.import_meta_graph(model_path + '.meta')
        except OSError:
            print("[generate_seq] model %s does not exist" % model_path)
            return seq_start

        saver.restore(sess, model_path)

        in_X = Dataset.encode_str(output_seq)
        in_X = np.array([in_X])  # "1 bach of 1 sequence"

        in_h_state = np.zeros([1, hStateSize * nLayers], dtype=np.float32)  # [ BATCHSIZE, INTERNALSIZE * NLAYERS]
        for i in range(length):
            y_preds_prob, next_in_h_state = sess.run(
                ['y_preds_prob:0', 'next_h_state_input:0'],
                feed_dict={
                    'X_train:0': in_X,
                    'h_state_input:0': in_h_state,
                    'dropout:0': 0., 'batch_size:0': 1
                }
            )
            in_h_state = next_in_h_state

            char = Dataset.peek_char_from_prob(y_preds_prob[-1], top_n=2)
            in_X = np.array([[char]])  # "1 bacth of 1 sequence of 1 char"

            output_seq += Dataset.decode_char(char)
        return output_seq

## 5. Gluing it all together

In [11]:
# This function will give life to our RNN
def tf_work_your_magic():
    tf.reset_default_graph() # we're gonna make magic more than once! XD

    tf_create_inputs()
    tf_create_architecture()
    tf_graph_forward_propagation()
    tf_graph_fullyconnected_softmax()
    tf_graph_training()


## Let the fun begin...

* Learning to "payar" (going The Martin Fierro's way).
* Learning to compose some music.
![](./images/pereyra_mate.jpg)

## 1. Let's learn how to _payar_!

![](./images/pereyra_guitar.jpg)

Aquí me pongo a cantar<br>
al compás de la vigüela,<br>
que el hombre que lo desvela<br>
una pena estrordinaria,<br>
como la ave solitaria<br>
con el cantar se consuela.<br>
<br>
Pido a los santos del cielo<br>
que ayuden mi pensamiento:<br>
...

## 1. Let's learn how to _payar_!
### 1.1. First things  first - preprocessing

[dataset/martinfierro](dataset/martinfierro/)

In [12]:
mfierro_path = "dataset/martinfierro/"
mfierro_files = mfierro_path + "*.txt"

# NOTE: We need to run this code below only once
Dataset.encode_files(mfierro_files, target_codec="utf8")
Dataset.normalize_files(mfierro_files)

[dataset] encoding file 'dataset/martinfierro/martinfierro.txt' from utf8 to utf8.
[dataset] encoding file 'dataset/martinfierro/martinfierrovuelta.txt' from utf8 to utf8.
[dataset] normalizing file 'dataset/martinfierro/martinfierro.txt'.
[dataset] normalizing file 'dataset/martinfierro/martinfierrovuelta.txt'.


[dataset/martinfierro](dataset/martinfierro/)

## 1. Let's learn how to _payar_!
### 1.2. Training

In [19]:
# making sure our RNN is alive
tf_work_your_magic()


train(mfierro_files, input("model name: "), n_epochs=10)

# NOTE: if it's taking too long, just press:
#       <Esc> and then <I> twice to interrupt
#       the process and save the model

model name: pepe10
[dataset] file 'dataset/martinfierro/martinfierro.txt' loaded.
[dataset] file 'dataset/martinfierro/martinfierrovuelta.txt' loaded.
[dataset] finished (208161 characters loaded).
[training] starting to learn :) ...
[training][epoch 1/10] characters processed: 25600
[training][epoch 2/10] characters processed: 230400
[training][epoch 3/10] characters processed: 435200
[training][epoch 4/10] characters processed: 640000
[training][epoch 5/10] characters processed: 844800
[training][epoch 6/10] characters processed: 1049600
[training][epoch 7/10] characters processed: 1254400
[training][epoch 8/10] characters processed: 1459200
[training][epoch 9/10] characters processed: 1664000
[training][epoch 10/10] characters processed: 1868800
[training] training finished (0.7 mins)
[training] model saved: trained_models/test/pepe10


## 1. Let's learn how to _payar_!
### 1.3. Payando :)

In [20]:
canto = generate_seq(
    model_path=input("Trained model path: "),
    seq_start=input("Sequence starts with: "),
    length=1000
)
print(canto)

Trained model path: trained_models/test/pepe10
Sequence starts with: aqui
INFO:tensorflow:Restoring parameters from trained_models/test/pepe10
aquillrreero
4::yee  ea e ee e ee ae eerr
6::y e eee e eee  ee eeaaa

1: e  eee  e ee e  ae e earoa

:  ee  aeaaa  e ae e a aaa

:  ae  ea  ea  ea ea eaaa

:: e  ee e ea  a e aaaaa

:  eea  ea e e e eaaaa

:: e e  e a  aa ea  aa aaaa
6: e  e ea  a  eaaaa eaaaa

:: e ee ea e eea  ae e aeao

:: ae  ee e a  a aaa e aara

: y   ea  aa a e ea  ee aea

1: ee  a e e ee  a  aa a aeo

1: ee  a aa e  e ea e e ee eao

:: a  ae e ae  a  aa ea aeo

:  e  a e  ea e ea e aa aea

:  a ea  a  e  a e  aa aaaa

:: ae  eaa  ea  e eaa ae eaa

:: a  e a e e  a  ea e a ara

:  e  a aa  a  ea e ea eaa

:  ee  e ea e e  a e aa aea

:  ae e  aaa  a e eaa eea

: ye eee e  a  a  e eaa aao

:  a e e  a  aa  ea  ea aaa

:  e  ea aa  eaa  a a aara

1: e e ea ea  a  aa  aa eeoa

:: ee ee e e ae e eea ea aea

:  e e e  e e  a  e ea e aeo

:  a  ae e e e  eeaa  eaaa

:  ee  eaaa

## 1. Let's learn how to _payar_!
### 1.4. Pretrained Payadores :D

In [21]:
from re import sub as re_replace

trained_epochs = [0, 50, 400]

In [24]:
trained_model = "trained_models/martinfierro/%depochs" % trained_epochs[1]

canto = generate_seq(model_path=trained_model, seq_start=input("Starts with: "), length=1000)

canto = re_replace(r'\d:\s', '', canto) # deleting all :1,:2,:3... etc.
print(canto)

Starts with:       
INFO:tensorflow:Restoring parameters from trained_models/martinfierro/400epochs
      manas
el ciego y las carazos,
y aqui no ma gan escura
tengan tener el cantar.

Yo no sey cantorecantor
a mes de las que es sucor,
yo no soy rentrando el como,y so es de tanta cantar:
si po dejar e algun rancho
se lo quise canta gartar.

Yl se coltido a todito
la gusta que lo se ponga,
yo se que pas conoce al
y conte hombre se mo el nen,
pero poco entre el sueno
farrigonos que hacia cruta.

A su cargue ten peligro,
parque le tienen se erieza:
ya no hay quien no por venia,
yo tiene que sobre estantes,
porque el pobre se dintraba
y su carguenta sin nadre.

Es triste en ma sal de mismo,
nunca le folta el peluero;
yo si lo que hay sanide
el hombre y el solo el pierto;
y ya de el ser premenio
lo medao que sabe diene.

A vaces decia al vieje
y esta la suerte no liera,
hicieron ciliar la pierra
con una cora certena.
Aujuna, pobre tidia,
no teme man que no sartida!

Si se alla a una mujer
q

## 2. Let's learn how to compose music!
### ( Now we're talking :D )
![](./images/abc.png)

### The ABC Notation (http://abcnotation.com/)

Example:
```
X: 1
T: Cooley's
M: 4/4
L: 1/8
R: reel
K: Emin
Q:120
|:D2|EB{c}BA B2 EB|~B2 AB dBAG|FDAD BDAD|FDAD dAFD|
EBBA B2 EB|B2 AB defg|afe^c dBAF|DEFD E2:|
|:gf|eB B2 efge|eB B2 gedB|A2 FA DAFA|A2 FA defg|
eB B2 eBgB|eB B2 defg|afe^c dBAF|DEFD E2:|

```

**Good tutorial:** [How to interpret abc music notation](http://www.lesession.co.uk/abc/abc_notation.htm)

### Playing ABC files


**Software:** http://abcnotation.com/software

**Webplayer:** https://abcjs.net/abcjs-editor.html


**But** we're  gonna be using these commands: [abcmidi](http://abc.sourceforge.net/abcMIDI/original/), [timidity](https://sfxpt.wordpress.com/2015/02/02/how-to-play-midi-files-under-ubuntu-linux/).

**Installation:**
```
~$ sudo apt install abcmidi
~$ sudo apt-get install timidity timidity-interfaces-extra
```
**Usage:**
```
~$ abc2midi song.abc -o song.mid
~$ timidity song.mid
```

## 2. Let's learn how to compose music!
### 2.1. Remember, first things  first - preprocessing

[dataset/music](dataset/music/)

In [25]:
music_path = "dataset/music/"
music_files = music_path + "**/*.[ta][xb][tc]"  # Recursively, all .abc or .txt files

# NOTE: We need to run this code below only once
Dataset.encode_files(music_files, target_codec="utf8")
Dataset.normalize_files(music_files)

[dataset] encoding file 'dataset/music/The Nottingham Collection/morris.txt' from utf8 to utf8.
[dataset] encoding file 'dataset/music/The Nottingham Collection/playford.txt' from utf8 to utf8.
[dataset] encoding file 'dataset/music/The Nottingham Collection/slip.txt' from utf8 to utf8.
[dataset] encoding file 'dataset/music/The Nottingham Collection/reelsa-c.txt' from utf8 to utf8.
[dataset] encoding file 'dataset/music/The Nottingham Collection/reelsu-z.txt' from utf8 to utf8.
[dataset] encoding file 'dataset/music/The Nottingham Collection/xmas.txt' from utf8 to utf8.
[dataset] encoding file 'dataset/music/The Nottingham Collection/reelsr-t.txt' from utf8 to utf8.
[dataset] encoding file 'dataset/music/The Nottingham Collection/reelsh-l.txt' from utf8 to utf8.
[dataset] encoding file 'dataset/music/The Nottingham Collection/hpps.txt' from utf8 to utf8.
[dataset] encoding file 'dataset/music/The Nottingham Collection/jigs.txt' from utf8 to utf8.
[dataset] encoding file 'dataset/music

[dataset] encoding file 'dataset/music/lotro1/grecas.abc' from utf8 to utf8.
[dataset] encoding file 'dataset/music/lotro1/badame.abc' from utf8 to utf8.
[dataset] encoding file 'dataset/music/lotro1/breeland.abc' from utf8 to utf8.
[dataset] encoding file 'dataset/music/lotro1/monty.abc' from utf8 to utf8.
[dataset] encoding file 'dataset/music/lotro1/metori.abc' from utf8 to utf8.
[dataset] encoding file 'dataset/music/lotro1/romwha.abc' from utf8 to utf8.
[dataset] encoding file 'dataset/music/lotro1/vanmoom.abc' from utf8 to utf8.
[dataset] encoding file 'dataset/music/lotro1/blofirm.abc' from utf8 to utf8.
[dataset] encoding file 'dataset/music/lotro1/ateam.abc' from utf8 to utf8.
[dataset] encoding file 'dataset/music/lotro1/blisom.abc' from utf8 to utf8.
[dataset] encoding file 'dataset/music/lotro1/yngjud.abc' from utf8 to utf8.
[dataset] encoding file 'dataset/music/lotro1/dreamon.abc' from utf8 to utf8.
[dataset] encoding file 'dataset/music/lotro1/bruman.abc' from utf8 to ut

[dataset] encoding file 'dataset/music/lotro1/syscho.abc' from utf8 to utf8.
[dataset] encoding file 'dataset/music/lotro1/cops.abc' from utf8 to utf8.
[dataset] encoding file 'dataset/music/lotro1/hashm.abc' from utf8 to utf8.
[dataset] encoding file 'dataset/music/lotro1/ledgoi.abc' from utf8 to utf8.
[dataset] encoding file 'dataset/music/lotro1/johwel.abc' from utf8 to utf8.
[dataset] encoding file 'dataset/music/lotro1/seinfeld.abc' from utf8 to utf8.
[dataset] encoding file 'dataset/music/lotro1/ff6kef.abc' from utf8 to utf8.
[dataset] encoding file 'dataset/music/lotro1/madlaim.abc' from utf8 to utf8.
[dataset] encoding file 'dataset/music/lotro1/meaiwo.abc' from utf8 to utf8.
[dataset] encoding file 'dataset/music/lotro1/superman.abc' from utf8 to utf8.
[dataset] encoding file 'dataset/music/lotro1/teaeve1.abc' from utf8 to utf8.
[dataset] encoding file 'dataset/music/lotro1/stargate.abc' from utf8 to utf8.
[dataset] encoding file 'dataset/music/lotro1/simsoun.abc' from utf8 to

[dataset] normalizing file 'dataset/music/The Nottingham Collection/morris.txt'.
[dataset] normalizing file 'dataset/music/The Nottingham Collection/playford.txt'.
[dataset] normalizing file 'dataset/music/The Nottingham Collection/slip.txt'.
[dataset] normalizing file 'dataset/music/The Nottingham Collection/reelsa-c.txt'.
[dataset] normalizing file 'dataset/music/The Nottingham Collection/reelsu-z.txt'.
[dataset] normalizing file 'dataset/music/The Nottingham Collection/xmas.txt'.
[dataset] normalizing file 'dataset/music/The Nottingham Collection/reelsr-t.txt'.
[dataset] normalizing file 'dataset/music/The Nottingham Collection/reelsh-l.txt'.
[dataset] normalizing file 'dataset/music/The Nottingham Collection/hpps.txt'.
[dataset] normalizing file 'dataset/music/The Nottingham Collection/jigs.txt'.
[dataset] normalizing file 'dataset/music/The Nottingham Collection/coll.txt'.
[dataset] normalizing file 'dataset/music/The Nottingham Collection/ashover.txt'.
[dataset] normalizing file 

[dataset] normalizing file 'dataset/music/lotro1/beaifi.abc'.
[dataset] normalizing file 'dataset/music/lotro1/startrek.abc'.
[dataset] normalizing file 'dataset/music/lotro1/ozzmot.abc'.
[dataset] normalizing file 'dataset/music/lotro1/wereno.abc'.
[dataset] normalizing file 'dataset/music/lotro1/aveaft.abc'.
[dataset] normalizing file 'dataset/music/lotro1/marsex.abc'.
[dataset] normalizing file 'dataset/music/lotro1/pinhey.abc'.
[dataset] normalizing file 'dataset/music/lotro1/irishwa.abc'.
[dataset] normalizing file 'dataset/music/lotro1/kinyou.abc'.
[dataset] normalizing file 'dataset/music/lotro1/u2ang.abc'.
[dataset] normalizing file 'dataset/music/lotro1/ozzido.abc'.
[dataset] normalizing file 'dataset/music/lotro1/redcan.abc'.
[dataset] normalizing file 'dataset/music/lotro1/mairis.abc'.
[dataset] normalizing file 'dataset/music/lotro1/amehors.abc'.
[dataset] normalizing file 'dataset/music/lotro1/silprom.abc'.
[dataset] normalizing file 'dataset/music/lotro1/ff8att.abc'.
[dat

...

[dataset] normalizing file 'dataset/music/lotro1/lastkiss.abc'.
[dataset] normalizing file 'dataset/music/lotro1/chibed.abc'.
[dataset] normalizing file 'dataset/music/lotro1/family.abc'.
[dataset] normalizing file 'dataset/music/lotro1/priwyn.abc'.
[dataset] normalizing file 'dataset/music/lotro1/shalat.abc'.
[dataset] normalizing file 'dataset/music/lotro1/aveunb.abc'.
[dataset] normalizing file 'dataset/music/lotro1/benny.abc'.
[dataset] normalizing file 'dataset/music/lotro1/davtyp.abc'.
[dataset] normalizing file 'dataset/music/lotro1/maubol.abc'.
[dataset] normalizing file 'dataset/music/lotro1/metnoth.abc'.
[dataset] normalizing file 'dataset/music/lotro1/bonmake.abc'.
[dataset] normalizing file 'dataset/music/lotro1/rolgim.abc'.
[dataset] normalizing file 'dataset/music/lotro1/hallowe.abc'.
[dataset] normalizing file 'dataset/music/lotro1/alibet.abc'.
[dataset] normalizing file 'dataset/music/lotro1/dam9cr.abc'.
[dataset] normalizing file 'dataset/music/lotro1/ledtan.abc'.
[dat

[dataset] normalizing file 'dataset/music/lotro1/kandus.abc'.
[dataset] normalizing file 'dataset/music/lotro1/jacill.abc'.
[dataset] normalizing file 'dataset/music/lotro1/haraxe.abc'.
[dataset] normalizing file 'dataset/music/lotro1/avebat.abc'.
[dataset] normalizing file 'dataset/music/lotro1/bliada.abc'.
[dataset] normalizing file 'dataset/music/lotro1/hesapira.abc'.
[dataset] normalizing file 'dataset/music/lotro1/enyfro.abc'.
[dataset] normalizing file 'dataset/music/lotro1/ozzsho.abc'.
[dataset] normalizing file 'dataset/music/lotro1/judnig.abc'.
[dataset] normalizing file 'dataset/music/lotro1/extbon.abc'.
[dataset] normalizing file 'dataset/music/lotro1/yngtri.abc'.
[dataset] normalizing file 'dataset/music/lotro1/u2chr.abc'.
[dataset] normalizing file 'dataset/music/lotro1/johgho.abc'.
[dataset] normalizing file 'dataset/music/lotro1/grefra.abc'.
[dataset] normalizing file 'dataset/music/lotro1/lormum.abc'.
[dataset] normalizing file 'dataset/music/lotro1/jackass.abc'.
[datas

[dataset] normalizing file 'dataset/music/lotro1/goingho.abc'.
[dataset] normalizing file 'dataset/music/lotro1/savtot.abc'.
[dataset] normalizing file 'dataset/music/lotro1/garmak.abc'.
[dataset] normalizing file 'dataset/music/lotro1/johima.abc'.
[dataset] normalizing file 'dataset/music/lotro1/grand.abc'.
[dataset] normalizing file 'dataset/music/lotro1/redund.abc'.
[dataset] normalizing file 'dataset/music/lotro1/forcol.abc'.
[dataset] normalizing file 'dataset/music/lotro1/shire.abc'.
[dataset] normalizing file 'dataset/music/lotro1/bonhol.abc'.
[dataset] normalizing file 'dataset/music/lotro1/pudshe.abc'.
[dataset] normalizing file 'dataset/music/lotro1/dtstr.abc'.
[dataset] normalizing file 'dataset/music/lotro1/merica.abc'.
[dataset] normalizing file 'dataset/music/lotro1/jamfire.abc'.
[dataset] normalizing file 'dataset/music/lotro1/nirall.abc'.
[dataset] normalizing file 'dataset/music/lotro1/godwaltz.abc'.
[dataset] normalizing file 'dataset/music/lotro1/Adder.abc'.
[dataset

[dataset] normalizing file 'dataset/music/lotro0/f51.abc'.
[dataset] normalizing file 'dataset/music/lotro0/f54.abc'.
[dataset] normalizing file 'dataset/music/lotro0/f313.abc'.
[dataset] normalizing file 'dataset/music/lotro0/f63.abc'.
[dataset] normalizing file 'dataset/music/lotro0/f272.abc'.
[dataset] normalizing file 'dataset/music/lotro0/bd.abc'.
[dataset] normalizing file 'dataset/music/lotro0/ai.abc'.
[dataset] normalizing file 'dataset/music/lotro0/f195.abc'.
[dataset] normalizing file 'dataset/music/lotro0/f323.abc'.
[dataset] normalizing file 'dataset/music/lotro0/f82.abc'.
[dataset] normalizing file 'dataset/music/lotro0/ra.abc'.
[dataset] normalizing file 'dataset/music/lotro0/f46.abc'.
[dataset] normalizing file 'dataset/music/lotro0/f184.abc'.
[dataset] normalizing file 'dataset/music/lotro0/f117.abc'.
[dataset] normalizing file 'dataset/music/lotro0/f110.abc'.
[dataset] normalizing file 'dataset/music/lotro0/g10.abc'.
[dataset] normalizing file 'dataset/music/lotro0/fc.

[dataset] normalizing file 'dataset/music/lotro0/f258.abc'.
[dataset] normalizing file 'dataset/music/lotro0/f226.abc'.
[dataset] normalizing file 'dataset/music/lotro0/f169.abc'.
[dataset] normalizing file 'dataset/music/lotro0/f267.abc'.
[dataset] normalizing file 'dataset/music/lotro0/f336.abc'.
[dataset] normalizing file 'dataset/music/lotro0/f111.abc'.
[dataset] normalizing file 'dataset/music/lotro0/f205.abc'.
[dataset] normalizing file 'dataset/music/lotro0/f254.abc'.
[dataset] normalizing file 'dataset/music/lotro0/f87.abc'.
[dataset] normalizing file 'dataset/music/lotro0/f153.abc'.
[dataset] normalizing file 'dataset/music/lotro0/f17.abc'.
[dataset] normalizing file 'dataset/music/lotro0/f166.abc'.
[dataset] normalizing file 'dataset/music/lotro0/aa.abc'.
[dataset] normalizing file 'dataset/music/lotro0/f343.abc'.


## 1. Let's learn how to compose music!
### 2.2. Training

In [ ]:
# making sure our RNN is alive
tf_work_your_magic()

train(music_files, input("model name: "), n_epochs=1)

# Warning: this dataset is 12,703,923 chars long...
#          so each epoch is going to take a while.

# NOTE1: if it's taking too long, just press:
#       <Esc> and then <I> twice to interrupt
#       the process and save the model

## 2. Let's dance!
### 2.3. Dancing like a robot (a tiny little baby robot) :D

But first, we need a DJ:

In [26]:
def play(str_abc):
    from os import system
    tmp_song = "_tmp_song.abc"
    with open(tmp_song, "w") as abc_song:
        abc_song.write(str_abc)

    system('bash play_abc.bash ' + tmp_song)

## 2. Let's dance!
### 2.3. Dancing like a robot (a tiny little baby robot) :D

In [ ]:
song = generate_seq(
    model_path=input("Trained model path: "),
    seq_start=input("Sequence starts with: "),
    length=1000
)
print(song)

play(song)
# NOTE: press <Esc> and then <I> twice to stop playing

## 2. Let's dance!
### 2.3. Dancing like a Pretrained Robot XD

In [27]:
from re import sub as re_replace

trained_epochs = [0, 1, 4, 8]

In [35]:
trained_model = "trained_models/music/%depochs" % trained_epochs[-1]

song = generate_seq(model_path=trained_model, seq_start="X:", length=1500)

print(song)

INFO:tensorflow:Restoring parameters from trained_models/music/8epochs
X:  1
T: Tredetting of Wenedorn
Z: Gradind of silestore
%  Transpose:--
L:1/4
Q: 120
K: C

[^A,/2z/8] ^A,/2 [^A,/2z/8] [^A,3/8z/8] [^A/2z/8] [^A,/2z/8] [^D,/2z/4]
[F/2z/4] [^A,3/8z/4] [^A/2z/8] [^A/2z/8] [^A,/2z/8] [^A,3/8z/4]
[^A,/2z/8] [^A/2z/8] [^A,/2z/8] [^D/2z/8] [^D,/2z/4] [F/2z/8]
[^A,/2z/8] [^D/2z/8] [^A,/8z/8] [^A,/2z/8] [^A/2z/8] [^A,3/8z/8] [^A/2z/8] [^D/2z/8] [^A,3/8z/8] [^A/2z/8] [^A/2z/8] [^A,/2z/8]
[^A,/2z/8] [^D/2z/8] [^A,3/8z/4] ^D/8 [^A,/2z/8] ^D/8 [^A/2^D,/2]
[^D/8z/8] [^A,/2z/4] [^A/2z/8] [^A,/2z/8] [^A/2z/8] [^A,/2z/8]
[^D/2z/4] [^D/2z/8] [^A,/2z/8] [^A/2z/8] [^A,/2z/8] [^A/2z/8]
[^D,/2z/8] [^A/2z/8] [^A,/2z/8] [^D/2z/8] ^D,/2 [^A/8^A,/2] [^A/2z/8]
[^A,/8z/8] [=D/2z/4] ^A,/8 [^D/2^A/2z/8] [^D,/2z/8] [^A/8z/8]
[^A,/2z/8] [^A/2z/8] [=D/2z/8] [^A,/2z/8] [^A/2z/8]
[^A,/2z/8] [^A/2z/8] [^A,/2z/8] [^A/8^A/2z/8] [^A,/2z/8]
[^A/8z/8] [^A,/2z/8] [^A/8z/8] [^A/2z/8] [^A,/8z/8] [^A,/2z/8]
[^A,/2z/8] [^A/2z

In [36]:
play(song) # (shake it)^n

# NOTE: press <Esc> and then <I> twice to stop playing

# Thanks for you attention!
### (...and that's it)
![](./images/robot-dancing.gif)